In [1]:
import pandas as pd
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f99573c2910> [RTYPES.NILSXP]

In [2]:
utterances = pd.read_csv("results/intelligibility/utterances.csv")
utterances.head()


,response_latency,response_latency_follow_up,age,corpus,transcript_file,child_name,utt_child,utt_car,utt_child_follow_up,utt_child_intelligible,utt_car_intelligible,follow_up_intelligible,caregiver_response,caregiver_response_contingent
0,-998.0,-186.0,36,MPI-EVA-Manchester,/home/mitja/data/CHILDES/MPI-EVA-Manchester/El...,MPI-EVA-Manchester_Eleanor,oops your paint,no mess please,Daddy there's all water on your computer,1,1,1,1,1
1,251.0,3895.0,36,MPI-EVA-Manchester,/home/mitja/data/CHILDES/MPI-EVA-Manchester/El...,MPI-EVA-Manchester_Eleanor,Daddy there's all water on your computer,NaN,I want five,1,0,1,0,0
2,108.0,5.0,36,MPI-EVA-Manchester,/home/mitja/data/CHILDES/MPI-EVA-Manchester/El...,MPI-EVA-Manchester_Eleanor,if I could . very good,what number are you doing now Poppin,um these,1,1,1,1,1
3,35.0,90.0,36,MPI-EVA-Manchester,/home/mitja/data/CHILDES/MPI-EVA-Manchester/El...,MPI-EVA-Manchester_Eleanor,um these,xxx,I know who's that,1,0,1,0,0
4,11.0,4681.0,36,MPI-EVA-Manchester,/home/mitja/data/CHILDES/MPI-EVA-Manchester/El...,MPI-EVA-Manchester_Eleanor,big one,well whichever one you've got,mm no,1,1,1,1,1


In [11]:
%%R -i utterances
library(lme4)

# Caregiver contingency
m_caregiver_contingency<-glmer('caregiver_response ~ utt_child_intelligible + (utt_child_intelligible | child_name)', data=utterances, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: caregiver_response ~ utt_child_intelligible + (1 | child_name)
   Data: utterances

      AIC       BIC    logLik  deviance  df.resid 
 316664.9  316696.7 -158329.5  316658.9    293980 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.4592  0.1341  0.5449  0.5693  1.4776 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.8454   0.9195  
Number of obs: 293983, groups:  child_name, 32

Fixed effects:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)             0.12819    0.10402   1.232    0.218    
utt_child_intelligible  1.00168    0.01799  55.679   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
utt_chld_nt -0.083


In [14]:
%%R -i utterances
library(lme4)

utterances_child_intelligible = subset(utterances, utt_child_intelligible==1)

# Child contingency
m_child_contingency<-glmer('follow_up_intelligible ~ caregiver_response + (caregiver_response | child_name)', data=utterances_child_intelligible, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_intelligible ~ caregiver_response + (caregiver_response |  
    child_name)
   Data: utterances_child_intelligible

     AIC      BIC   logLik deviance df.resid 
 88121.3  88174.0 -44055.6  88111.3   277266 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.7282  0.1658  0.1728  0.2159  1.9472 

Random effects:
 Groups     Name               Variance Std.Dev. Corr
 child_name (Intercept)        2.9022   1.7036       
            caregiver_response 0.2736   0.5231   0.64
Number of obs: 277271, groups:  child_name, 29

Fixed effects:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)          2.4413     0.1340  18.223  < 2e-16 ***
caregiver_response   0.4432     0.1068   4.152  3.3e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
crgvr_rsp